In [8]:
import pandas as pd

# Đường dẫn đến tệp dữ liệu CSV
file_path = 'NBR_Pixel_Values_Export.csv'

# Đọc tệp CSV vào DataFrame
data = pd.read_csv(file_path)
data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45173 entries, 0 to 45172
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  45173 non-null  object 
 1   NBR           45173 non-null  float64
 2   year          45173 non-null  int64  
 3   .geo          45173 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.4+ MB


In [44]:
data[['year_number', 'pixel_id']] = data['system:index'].str.split('_', expand=True)
data['year_number'] = data['year'].astype(int)
data['pixel_id'] = data['pixel_id'].astype(int)

# Chuyển đổi DataFrame để mỗi dòng là một pixel và mỗi cột là NBR của từng năm
pivot_df = data.pivot(index='pixel_id', columns='year_number', values='NBR')

# Đổi tên các cột để phản ánh năm theo thứ tự (Year_1, Year_2, ..., Year_38)
pivot_df.columns = [f'Year_{col}' for col in pivot_df.columns]

# Hiển thị kết quả
pivot_df.reset_index(inplace=True)
pivot_df.head()
pivot_df.to_csv('pivot_data.csv', index=False)

In [45]:
import xarray as xr
data = pivot_df.dropna()
years = [int(col.split('_')[1]) for col in data.columns if col.startswith("Year_")]
pixel_ids = data['pixel_id']

# Chuyển đổi DataFrame sang xarray.Dataset
ds = xr.Dataset(
    {
        "NBR": (["pixel_id", "year"], data.drop(columns="pixel_id").values)
    },
    coords={
        "pixel_id": pixel_ids,
        "time": years
    }
)

# Hiển thị Dataset
print(ds)

<xarray.Dataset>
Dimensions:   (pixel_id: 1365, year: 33, time: 33)
Coordinates:
  * pixel_id  (pixel_id) int32 0 1 2 3 4 5 6 ... 1359 1360 1361 1362 1363 1364
  * time      (time) int32 1985 1986 1987 1988 1989 ... 2013 2014 2015 2016 2017
Dimensions without coordinates: year
Data variables:
    NBR       (pixel_id, year) float64 -843.8 -806.3 -818.1 ... -788.2 -807.5


In [11]:
from dask.distributed import Client
# Khởi tạo Client để chạy trên local (tận dụng tất cả các lõi CPU sẵn có)
client = Client()
# Kiểm tra và hiển thị thông tin về cluster
print(client)

c:\ProgramData\anaconda3\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56220 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:56221' processes=4 threads=8, memory=15.80 GiB>


In [1]:
import dask.dataframe as dd
df = dd.read_csv('pivot_data.csv')
df.head()

,pixel_id,Year_1985,Year_1986,Year_1987,Year_1988,Year_1989,Year_1990,Year_1991,Year_1992,Year_1993,...,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017
0,0,-843.82715,-806.30030,-818.05650,-835.30310,-819.94190,-900.9146,-863.38420,-854.61395,-845.62310,...,-775.97750,-743.2432,-757.48615,-776.32030,-814.7087,-757.89470,-782.6427,-783.04820,-761.43790,-797.76900
1,1,-840.47920,-837.93225,-907.56616,-891.55206,-865.07935,-870.4422,-845.09937,-832.90580,-853.27870,...,-783.25806,-722.3077,-753.82320,-782.99120,-790.4526,-784.17267,-806.2424,-792.12600,-750.95300,-788.56525
2,2,-840.47920,-837.93225,-907.56616,-891.55206,-865.07935,-870.4422,-845.09937,-832.90580,-853.27870,...,-783.25806,-722.3077,-753.82320,-782.99120,-790.4526,-784.17267,-806.2424,-792.12600,-750.95300,-788.56525
3,3,-863.33856,-826.46277,-835.72455,-811.25090,-844.51610,-848.4398,-845.79900,-861.17140,-857.99286,...,-723.69684,-715.0072,-766.63110,-739.10060,-829.0749,-776.38890,-783.5564,-781.80475,-749.90530,-820.60030
4,4,-859.28705,-830.46440,-812.96490,-840.17755,-824.30730,-877.7005,-843.64606,-814.59290,-888.97700,...,-707.63336,-689.6774,-726.87225,-783.81757,-806.4171,-761.27650,-741.9556,-762.18940,-741.97864,-765.22504
